In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
data1 = pd.read_excel('/content/drive/MyDrive/DATA CHẤT LƯỢNG KHÔNG KHÍ/data của viện nhi/AQ_2010_2019_01july2024 (2).xlsx')
data = data1.dropna()
data.shape



(2403, 19)

In [ ]:
# Tính toán số lượng dòng cho từng phần
total_rows = len(data)
train_size = int(0.95 * total_rows)
test_size  = total_rows - train_size


# Chia DataFrame thành các tập huấn luyện, kiểm tra và kiểm định
data_train = data.iloc[:train_size]
data_test = data.iloc[train_size: ]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

import warnings

# Tắt cảnh báo SettingWithCopyWarning
pd.options.mode.chained_assignment = None
# Chuẩn hóa các cột giá dựa trên tập huấn luyện và áp dụng cho tập kiểm tra và kiểm định
scaler = MinMaxScaler(feature_range=(-1, 1))
columns = ['hn_dmo3a',	'hn_dmo3m',	'hn_dmno',	'hn_dmno2',	'hn_dmnox',	'hn_dmco',	'hn_dmpm10',	'hn_dmpm25',	'hn_dmpm1',	'hn_dmwisp',	'hn_dmso2',	'hn_dmtemp',	'hn_dmhum']
# columns = ['hn_dmno2',	'hn_dmnox',	'hn_dmco',	'hn_dmpm10',	'hn_dmpm25',	'hn_dmpm1',	'hn_dmwisp',	'hn_dmso2',	'hn_dmtemp',	'hn_dmhum']
data_train[columns] = scaler.fit_transform(data_train[columns])
data_test_copy=data_test[columns]
data_test[columns] = scaler.transform(data_test[columns])
train_data = data_train[columns].values
test_data = data_test[columns].values

# Hiển thị kích thước của các tập dữ liệu
print('Training data shape: {}'.format(train_data.shape))
print('Test data shape: {}'.format(test_data.shape))

test_data[29]

Training data shape: (2282, 13)
Test data shape: (121, 13)


array([-0.68135073, -0.67232149, -0.96480459, -0.58601113, -0.77904948,
       -0.47340132, -0.36302285, -0.22884974,  0.1963141 , -0.4649301 ,
       -0.43832004,  0.54296836, -0.00274918])

In [ ]:
seq_len = 14
X_train = []
for i in range(seq_len, len(train_data),7):
  X_train.append(train_data[i-seq_len:i])
X_train = np.array(X_train)

# Test data
X_test = []
for i in range(seq_len, len(test_data),7):
    X_test.append(test_data[i-seq_len:i])
X_test = np.array(X_test)

print('Training set shape', X_train.shape)
print('Testing set shape' ,X_test.shape)


Training set shape (324, 14, 13)
Testing set shape (16, 14, 13)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, timesteps, features):
        super(Autoencoder, self).__init__()
        self.mask_value = 0.0

        # Encoder
        self.lstm1 = nn.LSTM(features, timesteps, batch_first=True)
        self.lstm2 = nn.LSTM(timesteps, timesteps // 2, batch_first=True)

        # Decoder
        self.repeat_vector = nn.Linear(timesteps // 2, timesteps)  # Simulates RepeatVector
        self.lstm3 = nn.LSTM(timesteps // 2, timesteps, batch_first=True)
        self.lstm4 = nn.LSTM(timesteps, timesteps, batch_first=True)
        self.output_layer = nn.Linear(timesteps, features)


    def forward(self, x):
        # Masking (equivalent): Mask the input using a custom method
        x = torch.where(x != self.mask_value, x, torch.zeros_like(x))

        # Encoder
        x, _ = self.lstm1(x)
        x, (h_n, _) = self.lstm2(x)

        # RepeatVector
        x = h_n.repeat(x.size(1), 1, 1).permute(1, 0, 2)  # Repeat along the sequence dimension

        # Decoder
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x = self.output_layer(x)

        return x

# Define the Autoencoder
class AutoencoderModel:
    def __init__(self, verbose=1, epochs=50, batch_size=16, loss='mse', optimizer='adam'):
        self.verbose = verbose
        self.epochs = epochs
        self.batch_size = batch_size
        self.loss_fn = nn.MSELoss() if loss == 'mse' else None
        self.optimizer_fn = optim.Adam if optimizer == 'adam' else None

    def create_model(self, timesteps, features):
        model = Autoencoder(timesteps, features)
        return model

    def train_model(self, model, X_train):
        optimizer = self.optimizer_fn(model.parameters())
        loss_fn = self.loss_fn

        X_train = torch.tensor(X_train, dtype=torch.float32)
        dataset = torch.utils.data.TensorDataset(X_train, X_train)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        # Training Loop
        for epoch in range(self.epochs):
            for inputs, targets in dataloader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                loss.backward()
                optimizer.step()

            if self.verbose:
                print(f"Epoch {epoch+1}/{self.epochs}, Loss: {loss.item()}")

In [ ]:
timesteps = 14
features = 13
autoencoder_model = AutoencoderModel()
model = autoencoder_model.create_model(timesteps, features)
autoencoder_model.train_model(model, X_train)

# from torchsummary import summary
# model = autoencoder_model.create_model(timesteps, features)  # Tạo mô hình
# summary(model, (timesteps, features))  # Sử dụng tóm tắt trên mô hình


Epoch 1/50, Loss: 0.3147449791431427
Epoch 2/50, Loss: 0.20296607911586761
Epoch 3/50, Loss: 0.13696615397930145
Epoch 4/50, Loss: 0.09558500349521637
Epoch 5/50, Loss: 0.08115266263484955
Epoch 6/50, Loss: 0.12179145961999893
Epoch 7/50, Loss: 0.08485640585422516
Epoch 8/50, Loss: 0.08851424604654312
Epoch 9/50, Loss: 0.10269737988710403
Epoch 10/50, Loss: 0.09767941385507584
Epoch 11/50, Loss: 0.05802350491285324
Epoch 12/50, Loss: 0.05360474810004234
Epoch 13/50, Loss: 0.1193990707397461
Epoch 14/50, Loss: 0.06518997997045517
Epoch 15/50, Loss: 0.0977707952260971
Epoch 16/50, Loss: 0.09242425858974457
Epoch 17/50, Loss: 0.05591186136007309
Epoch 18/50, Loss: 0.07162003964185715
Epoch 19/50, Loss: 0.07774795591831207
Epoch 20/50, Loss: 0.04788719862699509
Epoch 21/50, Loss: 0.04445599019527435
Epoch 22/50, Loss: 0.043743833899497986
Epoch 23/50, Loss: 0.08222826570272446
Epoch 24/50, Loss: 0.05076299235224724
Epoch 25/50, Loss: 0.08500225096940994
Epoch 26/50, Loss: 0.046465978026390

In [ ]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    X_test = torch.tensor(X_test, dtype=torch.float32)
    predictions = model(X_test)


pred = predictions.numpy()
test = X_test.numpy()


print(predictions[2,0,:])
print(X_test[2,0,:])

tensor([-0.8173, -0.8115, -0.6733, -0.4527, -0.5929, -0.4636, -0.6672, -0.6278,
        -0.5375, -0.5225, -0.5150,  0.2942,  0.4402])
tensor([-0.7919, -0.7709, -0.8555, -0.6683, -0.7655, -0.4668, -0.7081, -0.6449,
        -0.3028, -0.7622, -0.5337,  0.5953,  0.2152])


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(test.reshape(-1),pred.reshape(-1))
mae = mean_absolute_error(test.reshape(-1),pred.reshape(-1))



print('MSE: ', mse)
print('MAE: ', mae)

MSE:  0.09383969
MAE:  0.18398651


In [ ]:
data2 = data1.copy()
data2.head()

,date,holiday,thang,nam,dow,weekend,hn_dmo3a,hn_dmo3m,hn_dmno,hn_dmno2,hn_dmnox,hn_dmco,hn_dmpm10,hn_dmpm25,hn_dmpm1,hn_dmwisp,hn_dmso2,hn_dmtemp,hn_dmhum
0,2010-01-01,1,1,2010,5,0,12.773337,19.155500,73.313637,37.073391,110.411583,2869.264160,82.683708,63.687817,53.553741,0.935593,22.951050,16.750,96.25
1,2010-01-02,0,1,2010,6,1,11.941775,21.280800,79.978386,37.754845,117.710373,3091.271973,53.249500,37.463985,30.909113,1.031725,21.106625,16.725,93.50
2,2010-01-03,0,1,2010,0,1,54.091862,66.220001,51.733109,52.447487,104.222008,2647.305420,98.332199,56.999287,44.949402,1.011433,28.034859,18.625,83.50
3,2010-01-04,0,1,2010,1,0,39.317726,60.505001,52.855743,60.220432,113.033836,2787.883301,76.243179,50.712963,41.189350,1.534541,20.061163,19.825,87.25
4,2010-01-05,0,1,2010,2,0,52.317215,74.869003,46.449150,50.686054,97.168846,2615.624268,48.950001,33.689999,28.730000,1.638597,10.465546,22.025,86.00


In [ ]:
columns1 = ['hn_dmo3a',	'hn_dmo3m',	'hn_dmno',	'hn_dmno2',	'hn_dmnox',	'hn_dmco',	'hn_dmpm10',	'hn_dmpm25',	'hn_dmpm1',	'hn_dmwisp',	'hn_dmso2',	'hn_dmtemp',	'hn_dmhum']
data3 = data2[columns1].fillna(0)
data3.isnull().sum()

,0
hn_dmo3a,0
hn_dmo3m,0
hn_dmno,0
hn_dmno2,0
hn_dmnox,0
hn_dmco,0
hn_dmpm10,0
hn_dmpm25,0
hn_dmpm1,0
hn_dmwisp,0


In [ ]:
scaler1 = MinMaxScaler(feature_range=(-1, 1))
data3[columns1] = scaler1.fit_transform(data3[columns1])
data4 = data3[columns1].values



In [ ]:
seq_len = 14
X_x = []
for i in range(seq_len, len(data3)):
  X_x.append(data4[i-seq_len:i])
X_x = np.array(X_x)
X_x

array([[[-9.54915551e-01, -9.42550159e-01, -2.63863262e-01, ...,
         -5.68720942e-01, -8.06695649e-02,  9.74358974e-01],
        [-9.57850612e-01, -9.36176109e-01, -1.96943014e-01, ...,
         -6.03380009e-01, -8.20416998e-02,  9.17948718e-01],
        [-8.09078732e-01, -8.01397591e-01, -4.80551730e-01, ...,
         -4.73189787e-01,  2.22405585e-02,  7.12820513e-01],
        ...,
        [-8.57342930e-01, -8.34882883e-01, -3.54608720e-01, ...,
         -4.89072937e-01, -1.94556768e-01,  3.38461538e-01],
        [-7.84018878e-01, -7.49818555e-01, -1.80899535e-01, ...,
          4.96452482e-02, -2.20627333e-01,  2.30769231e-01],
        [-8.18011958e-01, -7.38325885e-01, -1.70895688e-01, ...,
          5.25031042e-02, -1.94556768e-01,  2.51282051e-01]],

       [[-9.57850612e-01, -9.36176109e-01, -1.96943014e-01, ...,
         -6.03380009e-01, -8.20416998e-02,  9.17948718e-01],
        [-8.09078732e-01, -8.01397591e-01, -4.80551730e-01, ...,
         -4.73189787e-01,  2.22405585e

In [ ]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():  # Disable gradient calculation
    X1 = torch.tensor(X_x, dtype=torch.float32)
    predictions1 = model(X1)


pred1 = predictions1.numpy()

In [ ]:
x1 = []

for i in range(38):
    x1.append(pred1[0,i,:])

for i in range(1,pred1.shape[0],1):
    x1.append(pred1[i,29,:])

x1 = np.array(x1)
x1.shape
x1

IndexError: index 14 is out of bounds for axis 1 with size 14

In [ ]:
data_pred1 = pd.DataFrame(x1, columns = ['hn_dmo3a',	'hn_dmo3m',	'hn_dmno',	'hn_dmno2',	'hn_dmnox',	'hn_dmco',	'hn_dmpm10',	'hn_dmpm25',	'hn_dmpm1',	'hn_dmwisp',	'hn_dmso2',	'hn_dmtemp',	'hn_dmhum'])
data_original1 = scaler1.inverse_transform(data_pred1[columns1])



In [ ]:
data_test_original1 = pd.DataFrame(data_original1, columns=columns1)
data_test_original1

,hn_dmo3a,hn_dmo3m,hn_dmno,hn_dmno2,hn_dmnox,hn_dmco,hn_dmpm10,hn_dmpm25,hn_dmpm1,hn_dmwisp,hn_dmso2,hn_dmtemp,hn_dmhum
0,87.352570,95.323624,51.388046,52.397892,80.126350,2960.560791,101.939499,55.616013,64.798683,4.237912,26.120584,21.130991,68.040237
1,77.655869,90.584015,55.238846,53.606461,82.664894,2918.479980,87.798088,51.325172,58.261314,4.104486,16.966969,19.698540,68.417107
2,68.938812,79.332169,56.336254,54.310795,82.294266,2891.389160,81.185379,49.274124,58.338512,3.859992,15.343455,18.573769,68.301773
3,59.560524,67.899460,55.867664,54.254616,80.681259,2814.245605,74.828629,47.508228,58.937111,3.647501,14.917090,17.668983,68.249680
4,52.309326,59.381256,55.190685,54.056599,78.991562,2736.792480,69.415436,46.089390,59.467270,3.499619,14.675757,17.083601,68.247673
5,47.777748,54.621437,54.680634,53.928577,77.860382,2684.345459,65.618439,45.146648,59.748280,3.420125,14.391006,16.777029,68.304222
6,45.460754,52.686310,54.382267,53.870949,77.285988,2657.533447,63.332455,44.599880,59.796658,3.389204,14.098915,16.663515,68.378708
7,44.573166,52.362484,54.233810,53.843201,77.064621,2647.875488,62.117142,44.303051,59.684383,3.385215,13.850528,16.662785,68.443398
8,44.466000,52.760677,54.171337,53.818024,77.013222,2647.150879,61.548813,44.139832,59.484924,3.393028,13.666006,16.716894,68.487343
9,44.710423,53.365723,54.151245,53.785236,77.019913,2649.951660,61.327419,44.039490,59.254696,3.404089,13.542522,16.789793,68.510384


In [ ]:
columns1 = ['hn_dmo3a',	'hn_dmo3m',	'hn_dmno',	'hn_dmno2',	'hn_dmnox',	'hn_dmco',	'hn_dmpm10',	'hn_dmpm25',	'hn_dmpm1',	'hn_dmwisp',	'hn_dmso2',	'hn_dmtemp',	'hn_dmhum']
data5 = data2[columns1].fillna(0)

for i in columns:
  for j in range(3650):
    if data5[i][j] == 0:
      data5[i][j] = data_test_original1[i][j]
data5.isnull().sum()

<ipython-input-127-0168ad268f92>:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data5[i][j] = data_test_original1[i][j]


,0
hn_dmo3a,0
hn_dmo3m,0
hn_dmno,0
hn_dmno2,0
hn_dmnox,0
hn_dmco,0
hn_dmpm10,0
hn_dmpm25,0
hn_dmpm1,0
hn_dmwisp,0


In [ ]:
data_drop = data2.drop(columns=columns1)
data_drop.head()

,date,holiday,thang,nam,dow,weekend
0,2010-01-01,1,1,2010,5,0
1,2010-01-02,0,1,2010,6,1
2,2010-01-03,0,1,2010,0,1
3,2010-01-04,0,1,2010,1,0
4,2010-01-05,0,1,2010,2,0


In [ ]:
data_new = pd.concat([data_drop, data5], axis=1)
output_file = 'data_output.xlsx'  # Đặt tên file đầu ra
data_new.to_excel(output_file, index=False)
data_new.head()


,date,holiday,thang,nam,dow,weekend,hn_dmo3a,hn_dmo3m,hn_dmno,hn_dmno2,hn_dmnox,hn_dmco,hn_dmpm10,hn_dmpm25,hn_dmpm1,hn_dmwisp,hn_dmso2,hn_dmtemp,hn_dmhum
0,2010-01-01,1,1,2010,5,0,12.773337,19.155500,73.313637,37.073391,110.411583,2869.264160,82.683708,63.687817,53.553741,0.935593,22.951050,16.750,96.25
1,2010-01-02,0,1,2010,6,1,11.941775,21.280800,79.978386,37.754845,117.710373,3091.271973,53.249500,37.463985,30.909113,1.031725,21.106625,16.725,93.50
2,2010-01-03,0,1,2010,0,1,54.091862,66.220001,51.733109,52.447487,104.222008,2647.305420,98.332199,56.999287,44.949402,1.011433,28.034859,18.625,83.50
3,2010-01-04,0,1,2010,1,0,39.317726,60.505001,52.855743,60.220432,113.033836,2787.883301,76.243179,50.712963,41.189350,1.534541,20.061163,19.825,87.25
4,2010-01-05,0,1,2010,2,0,52.317215,74.869003,46.449150,50.686054,97.168846,2615.624268,48.950001,33.689999,28.730000,1.638597,10.465546,22.025,86.00
